## Step 0: Install docker and place sudo password in ~/.sudopassword

### Removed previous run containers (Skip when starting for first time)

In [10]:
!sudo -S docker rm /zookeeper-server < ~/.sudopassword
!sudo -S docker rm /kafka-server < ~/.sudopassword
!sudo -S docker rm /event-stream < ~/.sudopassword
!sudo -S docker network rm app-tier < ~/.sudopassword

[sudo] password for kalinga: /zookeeper-server
[sudo] password for kalinga: /kafka-server
[sudo] password for kalinga: /event-stream
[sudo] password for kalinga: app-tier


## Step 1: Create a network

In [11]:
!sudo -S docker network create app-tier --driver bridge < ~/.sudopassword

[sudo] password for kalinga: c3a62d975ed2a66bf7249ad7ad3ccd6211925a99ec07ec3ef3c8436b09eda3b7


## Step 2: Launch the Zookeeper server instance

In [12]:
!sudo -S  docker run -d --name zookeeper-server --network app-tier -e ALLOW_ANONYMOUS_LOGIN=yes bitnami/zookeeper:latest < ~/.sudopassword

[sudo] password for kalinga: c17d29a774368f71bca2e39f9a1779e2982abcb59fc0e152e89836eef7c75052


## Step 3: Launch the Kafka server instance

In [13]:
!sudo -S docker run -d --name kafka-server \
    --network app-tier \
    -e ALLOW_PLAINTEXT_LISTENER=yes \
    -e KAFKA_CFG_ZOOKEEPER_CONNECT=zookeeper-server:2181 \
    bitnami/kafka:latest < ~/.sudopassword

[sudo] password for kalinga: 502b6f7dadd232581b57b547558262bb00051a742b0c2e077c7699ced666d56a


## Step 4: Launch event-stream  instance

In [14]:
!sudo -S docker build -t kalinga/event-stream:v0.1 . < ~/.sudopassword

[sudo] password for kalinga: Sending build context to Docker daemon  48.64kB
Step 1/8 : FROM python:3
 ---> b162b5322f1c
Step 2/8 : LABEL Author="Kalinga Bhusan Ray"
 ---> Using cache
 ---> 55e95b0d90db
Step 3/8 : LABEL E-Mail="kalinga-bhusan.ray@tu-ilmenau.de"
 ---> Using cache
 ---> 5e490150ac8b
Step 4/8 : LABEL Desc="This is an image for event stream engiene"
 ---> Using cache
 ---> 430a9dc7ecdd
Step 5/8 : RUN apt-get update &&     apt-get install -y vim
 ---> Using cache
 ---> 63f97c80a8c5
Step 6/8 : ADD process_event_stream.py /
 ---> 5fe0c6805f34
Step 7/8 : RUN pip install faust typing_extensions
 ---> Running in 391f5d04fc59
  Created wheel for opentracing: filename=opentracing-1.3.0-cp38-none-any.whl size=18449 sha256=86cc0f8b5890e3820f13b5a34c9aafdb81b548ff5f1e78fa1647fad379c858f4
  Stored in directory: /root/.cache/pip/wheels/5a/c7/a8/55354b8928b16dafbcd73fb8fc33d7d55cf61ed0845b912ec8
  Created wheel for colorclass: filename=colorclass-2.2.0-cp38-none-any.whl size=19398 sha25

In [15]:
!sudo -S docker run -d \
                -it  \
                --name event-stream \
                --mount type=bind,source="$(pwd)"/.,target=/app \
                --network app-tier \
                kalinga/event-stream:v0.1 /bin/bash < ~/.sudopassword    

[sudo] password for kalinga: 4b9aeb32828f2ee72ec80df79a5e0e3c87f4ad7e00b1d479094e30da55a03f5b


In [16]:
!sudo -S docker ps  < ~/.sudopassword

[sudo] password for kalinga: CONTAINER ID        IMAGE                       COMMAND                  CREATED              STATUS                  PORTS                                    NAMES
4b9aeb32828f        kalinga/event-stream:v0.1   "/bin/bash"              5 seconds ago        Up Less than a second   80/tcp                                   event-stream
502b6f7dadd2        bitnami/kafka:latest        "/entrypoint.sh /run…"   About a minute ago   Up About a minute       9092/tcp                                 kafka-server
c17d29a77436        bitnami/zookeeper:latest    "/entrypoint.sh /run…"   About a minute ago   Up About a minute       2181/tcp, 2888/tcp, 3888/tcp, 8080/tcp   zookeeper-server


In [17]:
!ls

containerFS	    operations		     README.md		    unit-test-1
Dockerfile	    process_event_stream.py  stream-event-app-data  unit-test-2
event-stream.ipynb  __pycache__		     test.py


### For existing containers (During development after restarting the host machine)

In [1]:
!sudo -S docker start /zookeeper-server < ~/.sudopassword
!sudo -S docker start /kafka-server < ~/.sudopassword
!sudo -S docker start /event-stream < ~/.sudopassword

[sudo] password for kalinga: /zookeeper-server
[sudo] password for kalinga: /kafka-server
[sudo] password for kalinga: /event-stream


In [3]:
!sudo -S docker ps < ~/.sudopassword

[sudo] password for kalinga: CONTAINER ID        IMAGE                       COMMAND                  CREATED             STATUS              PORTS                                    NAMES
f1966e227cf6        kalinga/event-stream:v0.1   "/bin/bash"              31 hours ago        Up 15 seconds       80/tcp                                   event-stream
91792eec8d10        bitnami/kafka:latest        "/entrypoint.sh /run…"   31 hours ago        Up 24 seconds       9092/tcp                                 kafka-server
27f4dbfafd0d        bitnami/zookeeper:latest    "/entrypoint.sh /run…"   31 hours ago        Up 28 seconds       2181/tcp, 2888/tcp, 3888/tcp, 8080/tcp   zookeeper-server


#### [In separate terminals]
docker exec -it event-stream bash
cd app
faust -A process_event_stream worker -l info 

docker exec -it kafka-server bash  
kafka-console-producer.sh --broker-list 127.0.0.1:9092 --topic transaction --producer.config /opt/bitnami/kafka/conf/producer.properties


## Step 5: Unit tests

In [18]:
!sudo -S docker cp unit-test-1 kafka-server:/ < ~/.sudopassword

[sudo] password for kalinga: 

In [19]:
!sudo -S docker cp unit-test-2 kafka-server:/ < ~/.sudopassword

[sudo] password for kalinga: 

[kafka-server terminal]  
kafka-console-producer.sh --broker-list 127.0.0.1:9092 --topic transaction --producer.config /opt/bitnami/kafka/conf/producer.properties < unit-test-1


In [ ]:
kafka-console-producer.sh --broker-list 127.0.0.1:9092 --topic transaction --producer.config /opt/bitnami/kafka/conf/producer.properties < unit-test-1

### Step 6: Extend the requirements

#### Development shortcuts

In [7]:
!sudo -S docker run -d \
                -it  \
                --name event-stream \
                --mount type=bind,source="$(pwd)"/.,target=/app \
                --network app-tier \
                kalinga/event-stream:v0.1 /bin/bash < ~/.sudopassword    

[sudo] password for kalinga: db971051eb54966f51e53d7a1759410614a1f8cc75983b362ee06768ae146a34


In [4]:
#!sudo -S docker container ls --all < ~/.sudopassword

In [ ]:
!sudo -S docker cp event-stream:/process_event_stream.py .  < ~/.sudopassword

In [ ]:
!sudo -S chown kalinga:kalinga process_event_stream.py < ~/.sudopassword

In [2]:
!sudo -S docker stop event-stream < ~/.sudopassword

[sudo] password for kalinga: event-stream


In [6]:
!sudo -S docker rm /event-stream < ~/.sudopassword

[sudo] password for kalinga: /event-stream


In [8]:
!sudo -S docker stop /zookeeper-server < ~/.sudopassword
!sudo -S docker stop /kafka-server < ~/.sudopassword
!sudo -S docker stop /event-stream < ~/.sudopassword

[sudo] password for kalinga: /zookeeper-server
[sudo] password for kalinga: /kafka-server
[sudo] password for kalinga: 
